In [53]:
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
%matplotlib inline
from textblob.classifiers import NaiveBayesClassifier
import re,string

# Load Data Train

In [41]:
train = pd.read_csv("C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\CrawlData_1_v2.csv", delimiter=',', encoding='utf-8')

In [ ]:
#Data Train masih ada baris yg kosong

In [42]:
train

,Tweet,Label
0,@FirstMediaCares ini kenapa internet rumah say...,neg
1,NaN,NaN
2,@IndiHome cara berhenti berlangganan bagaimana...,net
3,NaN,NaN
4,@FirstMediaCares Telfon nmr di atas d suruh le...,neg
...,...,...
2044,"@FirstMediaCares klau emang boat mau puts, ken...",neg
2045,NaN,NaN
2046,@FirstMediaCares om tolong di cek ini wifi ken...,net
2047,NaN,NaN


In [43]:
#hapus baris kosong
if row in train:
    NewTrain = train.dropna()

In [45]:
NewTrain
NewTrain.to_csv("C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\CrawlData_1_v2.csv", index = False)

In [69]:
NewTrain['Tweet'][28]

'@IndiHome Jaringannya diperbagus dong kak.... Ini di daerah kayumanis 8, matraman jakarta timur kok jarungannya lem… https://t.co/nlYleUZvNw'

In [48]:
NewTrain = pd.DataFrame(NewTrain)
NewTrain["Tweet"] = NewTrain['Tweet'].astype(str) 

# Preprocessing Data

In [49]:
#NewTrain['Tweet'] = NewTrain['Tweet'].str.replace('http\S+', '') #hapus HTML Tag
#NewTrain['Tweet'] = NewTrain['Tweet'].str.replace('RT', '') #hapus HTML Tag
#NewTrain['Tweet'] = NewTrain['Tweet'].str.replace('<.*?>', '') #hapus HTML Tag
#NewTrain['Tweet'] = NewTrain['Tweet'].str.replace('@\S+', '') #Hapus Username
#NewTrain['Tweet'] = NewTrain['Tweet'].str.replace('\d+', '') #Hapus Angka
#NewTrain['Tweet'] = NewTrain['Tweet'].str.replace('[^\w\s]', '') #Hapus Spesial Karakter

#lowercase
#NewTrain['Tweet'] = NewTrain['Tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))

,Tweet,Label
0,@FirstMediaCares ini kenapa internet rumah say...,neg
2,@IndiHome cara berhenti berlangganan bagaimana...,net
4,@FirstMediaCares Telfon nmr di atas d suruh le...,neg
6,"@FirstMediaCares hi admin, sepertinya skrng su...",pos
8,ASSALAMUALAIKUM !!!! Nomer pelanggan 122114220...,net
...,...,...
2040,@FirstMediaCares min saya dengan nomor pelangg...,neg
2042,RT @zhiyou85: Happy Anniv @TelkomCare @IndiHom...,pos
2044,"@FirstMediaCares klau emang boat mau puts, ken...",neg
2046,@FirstMediaCares om tolong di cek ini wifi ken...,net


In [96]:
def preprocess(tweet):
	tweet = re.sub(r'RT','',tweet) #hapus tulisan "RT"
	tweet = re.sub(r'http\S+','',tweet) #hapus http
	tweet = re.sub(r'@[\w]*','',tweet)#tweet = re.sub(r'@\S+','',tweet) #hapus username
	tweet = hapus_tanda(tweet)
	tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #hapus #tagger
	tweet = re.sub(r'\w*\d\w*', '',tweet).strip()#hapus angka dan angka yang berada dalam string
	tweet = hapus_katadouble(tweet)#hapus repetisi karakter 
	tweet = tweet.lower()#ubah jadi lower case
	return tweet

def hapus_tanda(tweet): 
	tanda_baca = set(string.punctuation)
	tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
	return tweet

def hapus_katadouble(s): 
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def hapus_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
						   u"\U00002500-\U00002BEF"  # chinese char
						   u"\U00002702-\U000027B0"
						   u"\U00002702-\U000027B0"
						   u"\U000024C2-\U0001F251"
						   u"\U0001f926-\U0001f937"
						   u"\U00010000-\U0010ffff"
						   u"\u2640-\u2642"
						   u"\u2600-\u2B55"
						   u"\u200d"
						   u"\u23cf"
						   u"\u23e9"
						   u"\u231a"
						   u"\ufe0f"  # dingbats
						   u"\u3030"
						   u"\x01"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def kbbi(token): 
	#kbba = [kamus.strip('\n').strip('\r') for kamus in open('kamus\\kbba.txt')]
	kbba = [kamus.strip('\n').strip('\r') for kamus in open('I:\\PENTING\\analisis-sentimen-master\\analisis-sentimen-master\\kamus\\kbba.txt')]
	#ubah list menjadi dictionary 
	dic={}
	for i in kbba: 
		(key,val) = i.split('\t')
		dic[str(key)] = val
	#kbbi cocokan 
	final_string = ' '.join(str(dic.get(word, word)) for word in token).split()
	return final_string

def get_fitur(tweets):
	tokens = tweets.split()
	tokens = kbbi(tokens)
	return tokens

In [97]:
tweet_hslPre = []
for row in NewTrain['Tweet']:
    pre = preprocess(row)
    hapus_emoticon = hapus_emoji(pre) 
    fitur = get_fitur(hapus_emoticon)
    #tweet_hslPre = ' '.join(fitur)
    tweet_hslPre.append(' '.join(fitur))
    #print(fitur)

In [98]:
NewTrain['Tweet'] = tweet_hslPre

In [99]:
NewTrain

,Tweet,Label
0,ini kenapa internet rumah saya lambat banget i...,neg
2,cara berhenti berlangganan bagaimana caranya iya,net
4,telfon nomor di atas di suruh lewat twitter le...,neg
6,hai admin sepertinya sekarang sudah ok,pos
8,assalamualaikum nomer pelanggan,net
...,...,...
2040,admin saya dengan nomor pelanggan tiba tiba in...,neg
2042,bahagia anniv yang ke open all channel dari ta...,pos
2044,klau memang boat mau puts kena charge bilang d...,neg
2046,om tolong di cek ini wifi kenapa mati nyala no...,net


In [100]:
stop = stopwords.words('indonesian')
NewTrain['Tweet'] = NewTrain['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [101]:
NewTrain

,Tweet,Label
0,internet rumah lambat banget iya tolong cek id,neg
2,berhenti berlangganan iya,net
4,telfon nomor suruh twitter twitter suruh hubun...,neg
6,hai admin ok,pos
8,assalamualaikum nomer pelanggan,net
...,...,...
2040,admin nomor pelanggan internet tersambung moho...,neg
2042,bahagia anniv open all channel tanggal sd juli...,pos
2044,klau boat puts kena charge bilang sengaja bust...,neg
2046,om tolong cek wifi mati nyala nomer pelanggan ...,net


# Klasifikasi Data Train

In [109]:
myListDataTrain = []
for i in NewTrain:
    myListDataTrain = [tuple(x) for x in NewTrain.to_records(index=False)]
    

In [110]:
myListDataTrain

[('internet rumah lambat banget iya tolong cek id', 'neg'),
 ('berhenti berlangganan iya', 'net'),
 ('telfon nomor suruh twitter twitter suruh hubungi nomor', 'neg'),
 ('hai admin ok', 'pos'),
 ('assalamualaikum nomer pelanggan', 'net'),
 ('ngelag', 'neg'),
 ('all service tv internet turun jam mohon bantuan penyelesaiannya id pelanggan',
  'neg'),
 ('mohon cek dm nya admin', 'net'),
 ('berlangganan memakai telkomsel menyukseskan tujuan orang orang', 'net'),
 ('oi mati internetnya nomor pelanggan', 'neg'),
 ('berlangganan memakai telkomsel menyukseskan tujuan orang orang', 'net'),
 ('kasian hidupku penuh kebohongan lihat', 'net'),
 ('sepedanya merk iya', 'net'),
 ('smartcardnya dibersihin tetep admin teknisinya rumah', 'net'),
 ('jaringannya diperbagus kak daerah kayumanis matraman jakarta timur jaringannya lambat',
  'neg'),
 ('cek dm kak', 'net'),
 ('tolong benerin jaringannyaudah seminggu kaya ampe ijin kulen cm grgr wifi',
  'neg'),
 ('menit direstart modemnya', 'neg'),
 ('admin tol

In [111]:
kl = NaiveBayesClassifier(myListDataTrain)

# Loading TestingDataSet

In [113]:
NewTesting = pd.read_csv("C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\Data_Test_indihome.csv")

In [114]:
NewTesting

,Tweet
0,laku
1,m sinyal hilang
2,malam plis baca dm trimakasih sblmnya
3,hi indihome normal lambat spt kemaren komplain...
4,ttep ga internet gmn jadiny
...,...
193,makasii lho saa
194,om
195,hi ga lot gasih lg streaming tkem astaghfirullah
196,iya gitu mang sabar ganggu


In [119]:
myNewlistTestSet = []
for i in NewTesting.Tweet:
    myNewlistTestSet = [tuple(x) for x in NewTesting.to_records(index=False)]

In [120]:
myNewlistTestSet

[('laku',),
 ('m sinyal hilang',),
 ('malam plis baca dm trimakasih sblmnya',),
 ('hi indihome normal lambat spt kemaren komplain blm tanggap',),
 ('ttep ga internet gmn jadiny',),
 ('haha indihom aing mati kemaren',),
 ('punten rockstar blokir kah gabisa connect rockstar social club buka websitenya k',),
 ('emang indihome wkwk',),
 ('m sinyal hilang',),
 ('angka k jarak ga sampe km masuk akal',),
 ('gua maen dota aja ngestuck mulu ngntot',),
 ('tolong bantu follow up tiket krna bkom respon dr tugas lapang tks',),
 ('tagih indihome pdhal paket mas buanyak',),
 ('woi bro drakoran nape internet low gin siall',),
 ('emang biaya sewa modem ga dirincian tagih',),
 ('kamis sore udah whatsapp akun indihome teknisi jam siang tp',),
 ('maennya pake tenaga ya',),
 ('proses kak tedy',),
 ('sii malem gaada jaring mulu',),
 ('tolong kasi tmn kk sy kasi lg uji lot',),
 ('loh anak indihome ngomel mulu',),
 ('gua y',),
 ('m sinyal hilang',),
 ('gpp temen ngobrol hahaha',),
 ('indicum',),
 ('cepu anjro

In [121]:
for komen in myNewlistTestSet: #klasifikasi var mylistTesSet
    HslKlasifikasi = kl.classify(komen)
    print(HslKlasifikasi)

net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net
net


In [138]:
Testing2 = [('halo wifi rumah jam siang sampe malem gabisa dipake nih'),
           ('seru dan asik'),
           ('sudah normal kemabali'),
           ('internet lancar'),
           ('Terima kasih')]


In [139]:
for komen in Testing2: #klasifikasi var mylistTesSet
    HslKlasifikasi = kl.classify(komen)
    print(HslKlasifikasi)

neg
net
net
neg
net
